# import Pytorch
TODO
- train with smaller datasets
- train with smaller hidden_size

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# tokenizer
def installPyThaiNLP():
    !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
    !pip install epitran
    !pip install sklearn_crfsuite

In [ ]:
 installPyThaiNLP() # run once

  Using cached https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
  Created wheel for pythainlp: filename=pythainlp-2.3.0.dev0-cp36-none-any.whl size=11003368 sha256=0dc9c4f347f96ffca2a411357fce20b45c9adb8f13ad664cf03cfcfdf721df86
  Stored in directory: /tmp/pip-ephem-wheel-cache-x_erra0d/wheels/79/4e/1e/26f3198c6712ecfbee92928ed1dde923a078da3d222401cc78
Successfully built pythainlp


# Machine translation
## Handling Text



In [ ]:
def getEncoding(path):
    with open(path) as f:
        print(f.encoding)
getEncoding("nmt_data/generated_reviews_translator.csv")

UTF-8


In [ ]:
def readAndSeperateSourceAndTarget(path, encoding):
    data = pd.read_csv(path, encoding=encoding)
    return data["en_text"], data["th_text"]

source_sentence_raw, target_sentence_raw = readAndSeperateSourceAndTarget("nmt_data/generated_reviews_translator.csv", getEncoding("nmt_data/generated_reviews_translator.csv"))


UTF-8


In [ ]:
source_sentence_raw.head()

0    These are a terrible product from China.The li...
1    They can't be used for anything other than par...
2    And yes, I followed directions exactly as they...
3    What kind of trash can cannot survive simple h...
4                                  I'm returning them.
Name: en_text, dtype: object

In [ ]:
target_sentence_raw.head()

0    สินค้าพวกนี้เป็นสินค้าห่วยๆที่มาจากประเทศจีน ท...
1     ไม่สามารถใช้ทำอะไรได้มากกว่าใช้ประดับในงานเลี้ยง
2    และแน่นอนว่าฉันได้ทำตามแนวทางที่เขาเขียนไว้อย่...
3    กระป๋องขยะประเภทไหนกันที่ไม่สามารถใช้งานง่ายๆใ...
4                          ฉันจะส่งสินค้าพวกนี้กลับค่ะ
Name: th_text, dtype: object

Maps word into ID numbers

In [ ]:
class Vocabulary:
    def __init__(self):
        self.word2id = { "<s>": 0, "</s>": 1 }
        self.id2word = { 0: "<s>", 1: "</s>" }
        self.n_words = 2
        # with open(file_name) as f:
        #     for line in f:
        #         (word, id) = line.split()
        #         if int(id) == 1 or int(id) == 2:
        #             continue
        #         self.word2id[word] = int(id)
        #         self.id2word[int(id)] = word
        #         self.n_words += 1
    def getID(self, word):
        if word not in self.word2id:
            self.word2id[word] = self.n_words
            self.id2word[self.n_words] = word
            self.n_words += 1
        return self.word2id[word]

Tokenizer

In [ ]:
def runOnceNltk():
    import nltk
    nltk.download('punkt')
runOnceNltk()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from pythainlp import word_tokenize as tokenize_thai
from nltk.tokenize import word_tokenize as tokenize_eng
import re

def tokenizeThai(s):
    return tokenize_thai(s, keep_whitespace=False)
def tokenizeEng(s):
    return tokenize_eng(s)


In [ ]:
print(tokenizeThai(target_sentence_raw[0]))

['สินค้า', 'พวก', 'นี้', 'เป็น', 'สินค้า', 'ห่วย', 'ๆ', 'ที่', 'มาจาก', 'ประเทศ', 'จีน', 'ทั้ง', 'หลอดไฟ', 'และ', 'สกรู', 'เหล็ก', 'ที่', 'ที่', 'ใส่', 'เข้าไป', 'ใน', 'แก้ว', 'นั้น', 'ไม่', 'มี', 'แม้แต่', 'ความพอดี', '…', 'มัน', 'จึง', 'เกือบ', 'ที่จะ', 'เป็นไปไม่ได้', 'เลย', 'ที่', 'กระป๋อง', 'พวก', 'นี้', 'จะ', 'ไม่', 'แตก', 'ออก', 'เป็น', 'ชิ้น', 'เล็ก', 'ๆจน', 'ส่ง', 'ผลเสีย', 'ต่อ', 'อาหาร', 'ของ', 'คุณ']


In [ ]:
print(tokenize_eng(source_sentence_raw[0]))

['These', 'are', 'a', 'terrible', 'product', 'from', 'China.The', 'light', 'bulb', 'and', 'the', 'metal', 'screw', 'that', 'goes', 'into', 'the', 'glass', 'does', "n't", 'even', 'fit', 'properly', '...', 'so', 'it', 'is', 'almost', 'impossible', 'to', 'open', 'these', 'cans', 'without', 'breaking', 'a', 'very', 'small', 'part', 'off', 'or', 'risking', 'ruining', 'your', 'food', '.']


In [ ]:
def convertWord2id(data, tokenizer):
    vocab = Vocabulary()
    idData = []
    for index, sentence in data.items():
        sentence = addSOSandEOS(tokenizer(sentence))
        temp = []
        for word in sentence:
            temp.append(vocab.getID(word))
        idData.append(temp)
    return idData, vocab
def addSOSandEOS(sentence):
    sentence.insert(0, "<s>")
    sentence.append("</s>")
    return sentence

In [ ]:
prepared_target_sentence, target_vocab = convertWord2id(target_sentence_raw, tokenize_thai)
print(prepared_target_sentence[0])

[0, 2, 3, 4, 5, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 8, 8, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 8, 34, 3, 4, 35, 23, 36, 37, 5, 38, 39, 40, 41, 42, 43, 44, 45, 46, 1]


In [ ]:
prepared_source_sentence, source_vocab = convertWord2id(source_sentence_raw, tokenize_eng)
print(prepared_source_sentence[0])

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 12, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 4, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 1]



## Encoder-Decoder Approach (RNN with Attention)

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, hidden_size, max_length):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.max_length = max_length
        self.embedding = torch.nn.Embedding(vocab_size, hidden_size)
        self.gru = torch.nn.GRU(hidden_size, hidden_size)


    def forward(self, input, hidden):
        embedded = self.embedding(torch.tensor([[input]])).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden
    def process_sentence(self, input_sentence):
        """Processing of an entire input sentence"""
        hidden = torch.zeros(1, 1, self.hidden_size)
        encoder_outputs = torch.zeros(self.max_length, self.hidden_size)
        for i in range(len(input_sentence)):
            embedded = self.embedding(torch.tensor([[input_sentence[i]]])).view(1, 1, -1)
            output, hidden = self.gru(embedded, hidden)
            encoder_outputs[i] = output[0, 0]
        return encoder_outputs

In [ ]:
class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, hidden_size, max_length):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.gru = torch.nn.GRU(2 * hidden_size, hidden_size)
        self.embedding = torch.nn.Embedding(vocab_size, hidden_size)
        self.Wa = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.Ua = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.va = torch.nn.Parameter(torch.FloatTensor(1,hidden_size))
        self.out = torch.nn.Linear(3 * hidden_size, vocab_size)

    def forward(self, prev_output_id, prev_hidden, encoder_output,input_length):
        prev_output = self.embedding(torch.tensor([prev_output_id])).unsqueeze(1)
        
        m = torch.tanh(self.Wa(prev_hidden) + self.Ua(encoder_output))
        attention_scores = m.bmm(self.va.unsqueeze(2)).squeeze(-1)
        attention_scores = self.mask(attention_scores, input_length)
        attention_weights = torch.nn.functional.softmax( attention_scores, -1 )

        context = attention_weights.unsqueeze(1).bmm(encoder_output.unsqueeze(0))

        rnn_input = torch.cat((prev_output, context), 2)
        rnn_output, hidden = self.gru(rnn_input, prev_hidden)

        output = self.out(torch.cat((rnn_output, context, prev_output), 2))
        output = torch.nn.functional.log_softmax(output[0], dim=1)
        return output, hidden

    def mask(self, scores, input_length):
        s = scores.squeeze(0)
        for i in range(self.max_length-input_length):
            s[input_length+i] = -float('inf')
        return s.unsqueeze(0)
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [ ]:
def train(source_vocab, target_vocab, source_corpus, target_corpus):
    criterion = torch.nn.NLLLoss()
    hidden_size = 256
    learning_rate = 0.01

    # for visualize
    loss = []
    
    # initialize model
    encoder = Encoder(source_vocab.n_words, hidden_size, 100)
    decoder = Decoder(target_vocab.n_words, hidden_size, 100)
    encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.SGD(decoder.parameters(), lr=learning_rate)

    for epoch in range(10):
        total_loss_epoch = 0
        for source_sentence, target_sentence in zip(source_corpus, target_corpus):
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()

            encoder_output = encoder.process_sentence(source_sentence)

            sentence_loss = 0
            hidden = decoder.initHidden()
            for i in range(len(target_sentence)-1):
                output, hidden = decoder.forward(target_sentence[i], hidden, encoder_output, len(source_sentence))
                word_loss = criterion(output, torch.tensor([target_sentence[i+1]]))
                sentence_loss += word_loss

            total_loss_epoch += sentence_loss
            
            sentence_loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
        # loss.append(total_loss_epoch/len(source_corpus))

    return encoder, decoder, encoder_optimizer, decoder_optimizer, loss
        

In [ ]:
# ram ไม่พอ gg
trained_encoder, trained_decoder, _en_op, _de_op, loss = train(source_vocab, target_vocab, prepared_source_sentence, prepared_target_sentence)

In [ ]:
torch.save(trained_encoder.state_dict(), "models/encoder.pt")
torch.save(trained_decoder.state_dict(), "models/decoder.pt")